# Data Scraping

In [ ]:
from bs4 import BeautifulSoup
import requests
q=input("Enter the stock symbol: ")
url=f"https://merolagani.com/CompanyDetail.aspx?symbol={q}"
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0 Safari/537.36"
    )
}
response=requests.get(url,headers=headers)
soup=BeautifulSoup(response.content,'html.parser')
if response.status_code == 200:
    table=soup.find_all('tr')
    for row in table:
        columns=row.find_all(['th','td'])
        for column in columns:
            print(column.text.strip().replace('\n','').replace('\r',''),end=' ')
        print(" ")
else:
    print("Failed to retrieve data from the website.")

Sector Commercial Banks  
Shares Outstanding 270,569,970.00  
Market Price 499.30  
% Change 0.65 %  
Last Traded On 2026/02/05 02:59:54  
52 Weeks High - Low 562.00-471.00  
180 Day Average 506.96  
120 Day Average 502.04  
1 Year Yield 1.90%  
EPS 35.18                                                                    (FY:082-083, Q:2)  
P/E Ratio 14.19  
Book Value 235.64  
PBV 2.12  
% Dividend 12.50                                                                    (FY:081-082)  
#Fiscal YearValue1.                                                        12.50%                                                    (FY: 081-082)2.                                                        10.00%                                                    (FY: 080-081)3.                                                        11.00%                                                    (FY: 079-080)4.                                                        0.00%                                          

# Database Connection

In [ ]:
import dotenv
import psycopg2
config = dotenv.dotenv_values(".env")

host = config.get("DB_HOST") or os.environ.get("DB_HOST") or "localhost"
port = config.get("DB_PORT") or os.environ.get("DB_PORT") or "5432"
user = config.get("DB_USER") or os.environ.get("DB_USER") or "postgres"
password = config.get("DB_PASSWORD") or os.environ.get("DB_PASSWORD") or ""
database = config.get("DB_NAME") or os.environ.get("DB_NAME") or "postgres"

con = psycopg2.connect(
    host=host,
    port=port,
    user=user,
    password=password,
    database=database
)

python-dotenv could not parse statement starting at line 1
python-dotenv could not parse statement starting at line 2
python-dotenv could not parse statement starting at line 3
python-dotenv could not parse statement starting at line 4


KeyError: 'DB_HOST'